In [18]:
# Import pandas and yfinance
import pandas as pd
import yfinance as yf

# Import numpy 
import numpy as np
from numpy.linalg import multi_dot

# Set numpy random seed
np.random.seed(23)

# Import cufflinks
import cufflinks as cf
cf.set_config_file(offline=True, dimensions=((1000, 600)))

# Import plotly express for EF plot
import plotly.express as px
px.defaults.width, px.defaults.height = 1000, 600

import warnings 
warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)

In [6]:
# Specify assets/ stocks
# international etf portfolio: 'SPY', 'GLD', 'IWM', 'VWO', 'BND'

# indian stocks: bank, consumer goods, diversified, it, consumer durables
# ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']

assets = ['HDFCBANK', 'ITC', 'RELIANCE', 'TCS', 'ASIANPAINT']
assets.sort()

# Number of assets
numofasset = len(assets)

# Number of portfolio for optimization
numofportfolio = 5000

In [7]:
# Get yahoo tickers for indian stook
yahooticker = [x+'.NS' for x in assets]

# Fetch / read data for multiple stocks at once
df = yf.download(yahooticker, start='2015-01-01', end='2022-12-31', progress=False)['Adj Close']
df.columns = assets

# Write data to file for future use
df.to_csv('data/india_stocks.csv')

# Read from file
df = pd.read_csv('data/india_stocks.csv', index_col=0, parse_dates=True)

# Display dataframe
df

,ASIANPAINT,HDFCBANK,ITC,RELIANCE,TCS
Date,,,,,
2015-01-01,704.9196,446.3565,191.1796,417.9889,1065.1387
2015-01-02,729.2904,452.5686,191.7784,416.8826,1079.3239
2015-01-05,729.2433,448.7476,192.5334,412.3163,1062.9210
2015-01-06,711.8356,441.7619,187.5866,393.6035,1023.7346
2015-01-07,726.1381,443.0512,184.0979,402.1713,1011.6423
...,...,...,...,...,...
2022-12-26,3035.8335,1610.9755,327.3880,2524.0500,3156.5701
2022-12-27,3092.0093,1612.6068,325.9692,2544.7000,3162.9749
2022-12-28,3103.0359,1611.3215,327.1923,2544.4500,3160.6460


In [8]:
# Plot price history
df['2022':].normalize().iplot(kind='line')

In [9]:
# Dataframe of returns and volatility
returns = df.pct_change().dropna()
annual_returns = round(returns.mean()*260*100, 2)
annual_stdev = round(returns.std()*np.sqrt(260)*100, 2)

df1 = pd.DataFrame({
    'Ann Ret': annual_returns,
    'Ann Vol': annual_stdev
})
df1

,Ann Ret,Ann Vol
ASIANPAINT,22.98,26.88
HDFCBANK,19.71,23.75
ITC,10.63,27.00
RELIANCE,28.23,29.81
TCS,17.39,24.77


In [10]:
# Plot annualized return and volatility
df1.iplot(
    kind='bar',
    shared_xaxes=True,
    orientation='h'
)

In [11]:
df1.reset_index().iplot(
    kind='pie',
    labels='index',
    values='Ann Ret',
    textinfo='percent+label',
    hole=0.4
)

In [19]:
def portfolio_simulation(returns):
    
    # Initialize the lists
    rets = []; vols = []; wts = []
    
    # Simulate 5000 portfolios
    for i in range(numofportfolio):
        
        # Generate random weights
        weights = np.random.random(numofasset)
        
        # Set weights such that sum of weights equals 1
        weights /= np.sum(weights)
        
        # Portfolio statistics
        rets.append(weights.T @ np.array(returns.mean() * 260))
        vols.append(np.sqrt(multi_dot([weights.T, returns.cov()*260, weights])))
        wts.append(weights)
        
    # Create a dataframe for analysis
    data = {'port_rets': rets, 'port_vols': vols}
    for counter, symbol in enumerate(returns.columns.tolist ()):
        data[symbol+' weights'] = [w[counter] for w in wts]
        
    portdf = pd.DataFrame(data)
    portdf['sharpe_ratio'] = portdf['port_rets'] / portdf['port_vols']
    
    return round(portdf, 4)

In [20]:
# Create a data frame for analysis
temp = portfolio_simulation(returns)
temp.head()

,port_rets,port_vols,ASIANPAINT weights,HDFCBANK weights,ITC weights,RELIANCE weights,TCS weights,sharpe_ratio
0,0.1848,0.1760,0.1893,0.3465,0.2801,0.1033,0.0809,1.0498
1,0.2102,0.1805,0.3015,0.0734,0.1724,0.2716,0.1810,1.1650
2,0.1714,0.1775,0.0009,0.3322,0.3325,0.1129,0.2215,0.9660
3,0.2151,0.1855,0.3959,0.3420,0.0263,0.1193,0.1165,1.1599
4,0.1951,0.1769,0.3287,0.2503,0.0442,0.0002,0.3766,1.1028


In [21]:
# Get the max sharpe portfolio stats
temp.iloc[temp.sharpe_ratio.idxmax()]

port_rets             0.2231
port_vols             0.1835
ASIANPAINT weights    0.2465
HDFCBANK weights      0.2125
ITC weights           0.0131
RELIANCE weights      0.2893
TCS weights           0.2385
sharpe_ratio          1.2159
Name: 2108, dtype: float64

In [22]:
# Verify the above result
temp.describe().T

,count,mean,std,min,25%,50%,75%,max
port_rets,5000.0,0.1981,0.0167,0.1143,0.1869,0.1983,0.2095,0.2542
port_vols,5000.0,0.1809,0.0089,0.1675,0.1745,0.1791,0.1853,0.2523
ASIANPAINT weights,5000.0,0.2011,0.1122,0.0001,0.1148,0.2036,0.2777,0.7795
HDFCBANK weights,5000.0,0.1990,0.1138,0.0000,0.1093,0.1969,0.2773,0.6743
ITC weights,5000.0,0.2004,0.1151,0.0001,0.1091,0.1981,0.2778,0.9124
RELIANCE weights,5000.0,0.2016,0.1136,0.0002,0.1142,0.2006,0.2775,0.6545
TCS weights,5000.0,0.1978,0.1144,0.0002,0.1053,0.1964,0.2770,0.6886
sharpe_ratio,5000.0,1.0956,0.0798,0.4529,1.0544,1.1107,1.1538,1.2159


In [23]:
# Plot simulated portfolio
fig = px.scatter(
    temp, x='port_vols', y='port_rets', color='sharpe_ratio',
    labels={'port_vols': 'Expected Volatility', 'port_rets': 'Expected Return', 'sharpe_ratio': 'Sharpe Ratio'},
    title='Monte Carlo Simulated Portfolio'
).update_traces(mode='markers', marker=dict(symbol='cross'))

# Plot max sharpe
fig.add_scatter(
    mode='markers',
    x=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_vols']],
    y=[temp.iloc[temp.sharpe_ratio.idxmax()]['port_rets']],
    marker=dict(color='RoyalBlue', size=20, symbol='star'),
    name='Max Sharpe'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()

In [24]:
# Import optimization module from scipy
# sco.minimize?
import scipy.optimize as sco

In [25]:
def portfolio_stats(weights):
    
    weights = np.array(weights)
    port_rets = weights.T @ np.array(returns.mean() * 260)
    port_vols = np.sqrt(multi_dot([weights.T, returns.cov() * 260, weights]))
    
    return np.array([port_rets, port_vols, port_rets/port_vols])

# Minimize the volatility
def min_volatility(weights):
    return portfolio_stats(weights)[1]

# Minimize the variance
def min_variance(weights):
    return portfolio_stats(weights)[1]**2

# Maximize Sharpe ratio
def max_sharpe_ratio(weights):
    return -portfolio_stats(weights)[2]


In [26]:
# Specify constraints bounds and initial weights
cons = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
bnds = tuple((0, 1) for x in range(numofasset))
initial_wts = numofasset*[1./numofasset]

In [31]:
# Optimizing for maximum sharpe ratio
opt_sharpe = sco.minimize(max_sharpe_ratio, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

# Optimizing for minimum variance
opt_var = sco.minimize(min_variance, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)

In [32]:
opt_sharpe

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.2178090029703381
       x: [ 2.738e-01  2.293e-01  0.000e+00  2.793e-01  2.176e-01]
     nit: 6
     jac: [-3.827e-04  1.388e-04  3.090e-02  3.885e-05  2.855e-04]
    nfev: 36
    njev: 6

In [33]:
opt_var

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.0280400936325559
       x: [ 1.928e-01  2.367e-01  2.099e-01  7.286e-02  2.878e-01]
     nit: 7
     jac: [ 5.615e-02  5.592e-02  5.597e-02  5.613e-02  5.623e-02]
    nfev: 42
    njev: 7

In [34]:
# Efficient frontier
targetrets = np.linspace(0.155, 0.24, 100)
tvols = []

for tr in targetrets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: portfolio_stats(x)[0] - tr,
                'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    opt_ef = sco.minimize(min_volatility, initial_wts, method='SLSQP', bounds=bnds, constraints=cons)
    
    tvols.append(opt_ef['fun'])

targetvols = np.array(tvols)

In [36]:
# Create EF DataFrame for plotting
efport = pd.DataFrame({
    'targetrets': np.around(100*targetrets, 2),
    'targetvols': np.around(100*targetvols, 2),
    'targetsharpe': np.around(targetrets/targetvols, 2)
})

efport.head()

,targetrets,targetvols,targetsharpe
0,15.50,16.75,0.93
1,15.59,16.75,0.93
2,15.67,16.75,0.94
3,15.76,16.75,0.94
4,15.84,16.75,0.95


In [37]:
# Plot efficient frontier portfolio
fig = px.scatter(
    efport, x='targetvols', y='targetrets', color='targetsharpe',
    labels={'targetrets': 'Expected Return', 'targetvols': 'Expected Volatility', 'targetsharpe': 'Sharpe Ratio'},
    title='Efficient Frontier Portfolio'
).update_traces(mode='markers', marker=dict(symbol='cross'))


# Plot maximum sharpe portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]],
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='red', size=20, symbol='star'),
    name='Max Sharpe'
).update(layout_showlegend=False)

# Plot minimum variance portfolio
fig.add_scatter(
    mode='markers',
    x=[100*portfolio_stats(opt_var['x'])[1]],
    y=[100*portfolio_stats(opt_var['x'])[0]],
    marker=dict(color='green', size=20, symbol='star'),
    name='Min Variance'
).update(layout_showlegend=False)

# Show spikes
fig.update_xaxes(showspikes=True)
fig.update_yaxes(showspikes=True)
fig.show()